In [1]:
import os

import numpy as np
import importlib
import gymnasium as gym
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import patches
import matplotlib.animation as manimation

import gym_env
from models import LinearRL_NHB, SR_NHB
from utils import policy_reval, woodbury
from utils_render import plot_nhb_decisions, create_bar_plot

In [31]:
# Set the random seed for NumPy
seed = 42
np.random.seed(42)

# Save dir
save_dir = os.path.join('..', 'figures/')

In [4]:
# For plotting
prob_locs = [1, 2]
colors = [3, 2]

In [5]:
## Hyperparameters for individual plots to make results more similar to NHB
# This produces 0.46 and 0.65 = 0.19
# lambd = 10
# alpha = 0.1
# alpha_SR = 0.2
# beta = 0.9
# num_steps = 115
# num_iterations = 400

## Hyperparameters for plots all together to show a regular agent
# This produces 0.75 and 0.90 = 0.15
lambd = 10
alpha = 0.15
alpha_SR = 0.15
beta = 1.0
num_steps = 250
num_iterations = 400

# Policy Revaluation

## Run Simulations

### Complete Model

In [6]:
choices_complete = {"train":np.array([0,0]), "test":np.array([0,0])}
for i in range(num_iterations):
    # Define agent
    agent_complete =  LinearRL_NHB(_lambda=lambd, alpha=alpha, beta=beta, num_steps=num_steps, policy="softmax", imp_samp=True, exp_type="policy_reval")
    # Get optimal DR and update V and Z Values
    D_inv = agent_complete.get_D_inv()
    agent_complete.DR = D_inv
    agent_complete.update_Z()
    agent_complete.update_V()
    dec_train = np.argmax([agent_complete.V[1], agent_complete.V[2]])
    choices_complete["train"][dec_train] += 1

    # Update terminal state values
    agent_complete.update_exp()
    V_new, Z_new = policy_reval(agent_complete)
    dec_test = np.argmax([V_new[1], V_new[2]])
    choices_complete["test"][dec_test] += 1

### TD Models

In [7]:
choices_with = {"train":np.array([0,0]), "test":np.array([0,0])}
choices_without = {"train":np.array([0,0]), "test":np.array([0,0])}
choices_sr = {"train":np.array([0,0]), "test":np.array([0,0])}

for i in range(num_iterations):
    # Define agents
    agent_with_imp = LinearRL_NHB(_lambda=lambd, alpha=alpha, beta=beta, num_steps=num_steps, policy="softmax", imp_samp=True, exp_type="policy_reval")
    agent_without_imp = LinearRL_NHB(_lambda=lambd, alpha=alpha, beta=beta, num_steps=num_steps, policy="softmax", imp_samp=False, exp_type="policy_reval")
    agent_SR = SR_NHB(alpha=alpha_SR, beta=10.0, num_steps=num_steps, policy="softmax", exp_type="policy_reval")

    # Have the agents learn the environment
    agent_with_imp.learn(seed=i)
    agent_without_imp.learn(seed=i)
    agent_SR.learn(seed=i)
    
    ## SR
    # Original training policy
    dec_train = np.argmax([agent_SR.V[1], agent_SR.V[2]])
    choices_sr["train"][dec_train] += 1
    # Policy revaluation
    agent_SR.update_exp()
    agent_SR.update_V()
    # New policy
    dec_test = np.argmax([agent_SR.V[1], agent_SR.V[2]])
    choices_sr["test"][dec_test] += 1


    ## With importance sampling
    # Original training policy
    dec_train = np.argmax([agent_with_imp.V[1], agent_with_imp.V[2]])
    choices_with["train"][dec_train] += 1
    # Policy revaluation
    agent_with_imp.update_exp()
    V_new, Z_new = policy_reval(agent_with_imp)
    # New policy
    dec_test = np.argmax([V_new[1], V_new[2]])
    choices_with["test"][dec_test] += 1

    ## Without importance sampling
    # Original training policy
    dec_train = np.argmax([agent_without_imp.V[1], agent_without_imp.V[2]])
    choices_without["train"][dec_train] += 1
    # Policy revaluation
    agent_without_imp.update_exp()
    V_new, Z_new = policy_reval(agent_without_imp)
    # New policy
    dec_test = np.argmax([V_new[1], V_new[2]])
    choices_without["test"][dec_test] += 1


In [8]:
print(choices_with["train"])
print(f"selected state 1: {choices_with['test'][0]/num_iterations}\n")
print(choices_without)
print(f"selected state 1: {choices_without['test'][0]/num_iterations}")

[  0 400]
selected state 1: 0.755

{'train': array([  1, 399]), 'test': array([182, 218])}
selected state 1: 0.455


## Action Selection Probs

### Complete

In [9]:
prob_policy_complete = choices_complete["test"] / num_iterations
p_comp = choices_complete["test"][1] / num_iterations
std_policy_complete = np.sqrt((p_comp * (1 - p_comp)) / num_iterations)

### SR

In [10]:
prob_policy_sr = choices_sr["test"] / num_iterations
p_sr = choices_sr["test"][1] / num_iterations
std_policy_sr = np.sqrt((p_sr * (1 - p_sr)) / num_iterations)

### With Importance Sampling

In [11]:
prob_policy_with = choices_with["test"] / num_iterations

p_test = choices_with["test"][1] / num_iterations
std_policy_with = np.sqrt((p_test * (1 - p_test)) / num_iterations)

### Without Importance Sampling

In [12]:
prob_policy_without = choices_without["test"] / num_iterations

p_test = choices_without["test"][1] / num_iterations
std_policy_without = np.sqrt((p_test * (1 - p_test)) / num_iterations)

# Reward Revaluation

## Run Simulations

### Complete Model

In [13]:
choices_complete = {"train":np.array([0,0]), "test":np.array([0,0])}
for i in range(num_iterations):
    # Define agent
    agent_complete =  LinearRL_NHB(_lambda=lambd, alpha=alpha, beta=beta, num_steps=num_steps, policy="softmax", imp_samp=True, exp_type="reward_reval")
    # Get optimal DR and update V and Z Values
    D_inv = agent_complete.get_D_inv()
    agent_complete.DR = D_inv
    agent_complete.update_Z()
    agent_complete.update_V()
    dec_train = np.argmax([agent_complete.V[1], agent_complete.V[2]])
    choices_complete["train"][dec_train] += 1

    # Update terminal state values
    agent_complete.update_exp()
    V_new, Z_new = policy_reval(agent_complete)
    dec_test = np.argmax([V_new[1], V_new[2]])
    choices_complete["test"][dec_test] += 1

### TD Models

In [14]:
choices_with = {"train":np.array([0,0]), "test":np.array([0,0])}
choices_without = {"train":np.array([0,0]), "test":np.array([0,0])}
choices_sr = {"train":np.array([0,0]), "test":np.array([0,0])}

for i in range(num_iterations):
    # Define agents
    agent_with_imp = LinearRL_NHB(_lambda=lambd, alpha=alpha, beta=beta, num_steps=num_steps, policy="softmax", imp_samp=True, exp_type="reward_reval")
    agent_without_imp = LinearRL_NHB(_lambda=lambd, alpha=alpha, beta=beta, num_steps=num_steps, policy="softmax", imp_samp=False, exp_type="reward_reval")
    agent_SR = SR_NHB(alpha=alpha_SR, beta=10.0, num_steps=num_steps, policy="softmax", exp_type="reward_reval")

    # Have the agents learn the environment
    agent_with_imp.learn(seed=i)
    agent_without_imp.learn(seed=i)
    agent_SR.learn(seed=i)

    ## SR
    # Original training policy
    dec_train = np.argmax([agent_SR.V[1], agent_SR.V[2]])
    choices_sr["train"][dec_train] += 1
    # Policy revaluation
    agent_SR.update_exp()
    agent_SR.update_V()
    # New policy
    dec_test = np.argmax([agent_SR.V[1], agent_SR.V[2]])
    choices_sr["test"][dec_test] += 1
    
    ## With importance sampling
    # Original training policy
    dec_train = np.argmax([agent_with_imp.V[1], agent_with_imp.V[2]])
    choices_with["train"][dec_train] += 1
    # Policy revaluation
    agent_with_imp.update_exp()
    V_new, Z_new = policy_reval(agent_with_imp)
    # New policy
    dec_test = np.argmax([V_new[1], V_new[2]])
    choices_with["test"][dec_test] += 1

    ## Without importance sampling
    # Original training policy
    dec_train = np.argmax([agent_without_imp.V[1], agent_without_imp.V[2]])
    choices_without["train"][dec_train] += 1
    # Policy revaluation
    agent_without_imp.update_exp()
    V_new, Z_new = policy_reval(agent_without_imp)
    # New policy
    dec_test = np.argmax([V_new[1], V_new[2]])
    choices_without["test"][dec_test] += 1

In [15]:
print(choices_with["train"])
print(f"selected state 1: {choices_with['test'][0]/num_iterations}\n")
print(choices_without)
print(f"selected state 1: {choices_without['test'][0]/num_iterations}")

[  0 400]
selected state 1: 0.9025

{'train': array([  0, 400]), 'test': array([308,  92])}
selected state 1: 0.77


## Action Selection Probs

### Complete

In [16]:
prob_reward_complete = choices_complete["test"] / num_iterations
p_comp = choices_complete["test"][1] / num_iterations
std_reward_complete = np.sqrt((p_comp * (1 - p_comp)) / num_iterations)

### SR

In [17]:
prob_reward_sr = choices_sr["test"] / num_iterations
p_sr = choices_sr["test"][1] / num_iterations
std_reward_sr = np.sqrt((p_sr * (1 - p_sr)) / num_iterations)

### With Importance Sampling

In [18]:
prob_reward_with = choices_with["test"] / num_iterations

p_test = choices_with["test"][1] / num_iterations
std_reward_with = np.sqrt((p_test * (1 - p_test)) / num_iterations)

### Without Importance Sampling

In [19]:
prob_reward_without = choices_without["test"] / num_iterations

p_test = choices_without["test"][1] / num_iterations
std_reward_without = np.sqrt((p_test * (1 - p_test)) / num_iterations)

# Transition Revaluation

## Run Simulations

In [20]:
choices_with = {"train":np.array([0,0]), "test":np.array([0,0])}
choices_without = {"train":np.array([0,0]), "test":np.array([0,0])}

for i in range(num_iterations):
    # Define agents
    agent_with_imp = LinearRL_NHB(_lambda=lambd, alpha=alpha, beta=beta, num_steps=num_steps, policy="softmax", imp_samp=True, exp_type="trans_reval")
    agent_without_imp = LinearRL_NHB(_lambda=lambd, alpha=alpha, beta=beta, num_steps=num_steps, policy="softmax", imp_samp=False, exp_type="trans_reval")

    # Have the agents learn the environment
    agent_with_imp.learn(seed=i)
    agent_without_imp.learn(seed=i)

    ## With importance sampling
    # Original training policy
    dec_train = np.argmax([agent_with_imp.V[1], agent_with_imp.V[2]])
    choices_with["train"][dec_train] += 1

    # Transition revaluation
    T_new = agent_with_imp.construct_T_new()
    D_new = woodbury(agent=agent_with_imp, T=T_new, inv=False)
    agent_with_imp.DR = D_new
    agent_with_imp.update_Z()
    agent_with_imp.update_V()
    V_new = agent_with_imp.V

    # New policy
    dec_test = np.argmax([V_new[1], V_new[2]])
    choices_with["test"][dec_test] += 1

    ## Without importance sampling
    # Original training task
    dec_train = np.argmax([agent_without_imp.V[1], agent_without_imp.V[2]])
    choices_without["train"][dec_train] += 1

    # Transition revaluation
    T_new = agent_without_imp.construct_T_new()
    D_new = woodbury(agent=agent_without_imp, T=T_new, inv=False)
    agent_without_imp.DR = D_new
    agent_without_imp.update_Z()
    agent_without_imp.update_V()
    V_new = agent_without_imp.V

    # New policy
    dec_test = np.argmax([V_new[1], V_new[2]])
    choices_without["test"][dec_test] += 1

/Users/abizzle/Research/LinearRL-TD/src/models.py:350: RuntimeWarning: invalid value encountered in log
  self.V = np.round(np.log(self.Z), 2)


In [21]:
print(choices_with["train"])
print(f"selected state 1: {choices_with['test'][0]/num_iterations}\n")
print(choices_without)
print(f"selected state 1: {choices_without['test'][0]/num_iterations}")

[  0 400]
selected state 1: 0.7625

{'train': array([  0, 400]), 'test': array([110, 290])}
selected state 1: 0.275


## Action Selection Probs

### With Importance Sampling

In [22]:
prob_transition_with = choices_with["test"] / num_iterations

p_test = choices_with["test"][1] / num_iterations
std_transition_with = np.sqrt((p_test * (1 - p_test)) / num_iterations)

### Without Importance Sampling

In [23]:
prob_transition_without = choices_without["test"] / num_iterations

p_test = choices_without["test"][1] / num_iterations
std_transition_without = np.sqrt((p_test * (1 - p_test)) / num_iterations)

# Plot Simulation Results

## With Importance Sampling

In [43]:
# save_path = save_dir + "nhb_simulations_with.png"
save_path = None

# plot_nhb_decisions(
#     probs_reward=prob_reward_with,
#     probs_policy=prob_policy_with,
#     probs_transition=prob_transition_with,
#     colors=colors,
#     leg_loc='upper right',
#     save_path=save_path,
#     title='With Importance Sampling',
#     std=[std_reward_with, std_policy_with, std_transition_with]
# )

## Without Importance Sampling

In [42]:
# save_path = save_dir + "nhb_simulations_without.png"
save_path = None

# plot_nhb_decisions(
#     probs_reward=prob_reward_without,
#     probs_policy=prob_policy_without,
#     probs_transition=prob_transition_without,
#     colors=colors,
#     leg_loc='upper right',
#     save_path=save_path,
#     title='Without Importance Sampling',
#     std=[std_reward_without, std_policy_without, std_transition_without]
# )

# Plot NHB values against ours

## NHB reconstruction

In [38]:
# save_path = save_dir + "NHB_data.png"
save_path = None
nhb_means = [0.66, 0.47]
nhb_stds = [0.05, 0.05]
xlabels = ['Reward\nrevaluation', 'Policy\nrevaluation']
colors = [1, 9]
ylabel = 'Proportion of participants\nwho changed preference'

# Call the function
# create_bar_plot(nhb_means, colors, ylabel, xlabels, std=nhb_stds, save_path=save_path, title="Data")

## With Importance Sampling

In [39]:
# save_path = save_dir + "NHB_with_imp.png"
save_path = None
means = [prob_reward_with[0], prob_policy_with[0]]
stds = [std_reward_with, std_policy_with]
xlabels = ['Reward\nrevaluation', 'Policy\nrevaluation']
colors = [1, 9]
ylabel = 'Probability'

# Call the function
# create_bar_plot(means, colors, ylabel, xlabels, std=stds, save_path=save_path, title="Importance Sampling")

## SR

In [40]:
# save_path = save_dir + "NHB_SR.png"
save_path = None
means = [prob_reward_sr[0], prob_policy_sr[0]]
stds = [std_reward_sr, std_policy_sr]
xlabels = ['Reward\nrevaluation', 'Policy\nrevaluation']
colors = [1, 9]
ylabel = 'Probability'

# Call the function
# create_bar_plot(means, colors, ylabel, xlabels, std=stds, save_path=save_path, title="SR")

## Complete Model

In [41]:
# save_path = save_dir + "NHB_Complete.png"
save_path = None
means = [prob_reward_complete[0], prob_policy_complete[0]]
stds = [std_reward_complete, std_policy_complete]
xlabels = ['Reward\nrevaluation', 'Policy\nrevaluation']
colors = [1, 9]
ylabel = 'Probability'

# Call the function
# create_bar_plot(means, colors, ylabel, xlabels, std=stds, save_path=save_path, title="Complete")